# HOTRG

We implement the 2D HOTRG algorithm introduced in [Phys. Rev. B 86, 045139 (2012)](https://journals.aps.org/prb/abstract/10.1103/PhysRevB.86.045139). 

In [1]:
# using Pkg
# Pkg.add("TensorOperations")
# Pkg.add("Einsum")
using TensorOperations
using Einsum
using LinearAlgebra
using Printf

In [2]:
function create_w(temperature, field)
    b = 1 / temperature
    c = sqrt(cosh(b))
    s = sqrt(sinh(b))
    e1 = exp(b * field / 4)
    e2 = exp(- b * field / 4)
    w = [c * e1 s * e1; c * e2 -s * e2]
    return w
end;

Initializing local tensor and the impurity tensor. 

In [3]:
function initialize_bulk_tensor(temperature, field)
    w = create_w(temperature, field)
    @einsum ten_t[i,j,k,l] := w[a,i] * w[a,j] * w[a,k] * w[a,l]
    return ten_t
end;

function initialize_imp_t(temperature, field)
    w = create_w(temperature, field)
    s = [1, -1]
    @einsum imp_t[i,j,k,l] := s[a] * w[a,i] * w[a,j] * w[a,k] * w[a,l]
    return imp_t
end;

Calculating matrices which are used for obtaining projectors in the next step. 

In [4]:
function create_mm_left(t_up, t_down)
    @tensor mm[:] := t_up[-1, 1, 2, 5] * t_down[-2, 3, 5, 4] * t_up[-3, 1, 2, 6] * t_down[-4, 3, 6, 4]
    d1 = size(t_up)[1]
    d2 = size(t_down)[1]
    return reshape(mm, (d1 * d2, d1 * d2))
end;

In [5]:
function create_mm_right(t_up, t_down)
    @tensor mm[:] := t_up[1, -1, 2, 5] * t_down[3, -2, 5, 4] * t_up[1, -3, 2, 6] * t_down[3, -4, 6, 4]
    d1 = size(t_up)[1]
    d2 = size(t_down)[1]
    return reshape(mm, (d1 * d2, d1 * d2))
end;

Calculating the optimal projector (left or right). This step includes truncation.

In [6]:
function create_hotrg_projector_2d(dim, t_up, t_down)
    mm_left = create_mm_left(t_up, t_down)
    mm_right = create_mm_right(t_up, t_down)
    F = svd(mm_left)
    s1 = F.S
    u1 = F.U
    error_left = 0.
    for i in (dim + 1):length(s1)
        error_left += s1[i]
    end
    F = svd(mm_right)
    s2 = F.S
    u2 = F.U
    error_right = 0.
    for i in (dim + 1):length(s2)
        error_right += s2[i]
    end
    if error_left <= error_right
        dim_new = (length(s1) > dim) ? dim : length(s1)
        return (u1[:, 1:dim_new], s1)
    else
        dim_new = (length(s2) > dim) ? dim : length(s2)
        return (u2[:, 1:dim_new], s2)
    end
end;

Updating the local tensor in an optimized way (theoretical computational cost $O(D^7)$, memory cost $O(D^4)$).

In [7]:
function update_optimized(t_up, t_down, u_left, u_right)
    dim_hor = size(u_left)[2]
    dim_ver = size(t_up)[4]
    dx_up = size(t_up)[1]
    dx_down = size(t_down)[1]
    u_left = reshape(u_left, (dx_up, dx_down, dim_hor))
    u_right = reshape(u_right, (dx_up, dx_down, dim_hor))
    res = zeros(dim_hor, dim_hor, dim_ver, dim_ver)
    for a in 1:dim_ver
        upper = t_up[:,:,:,a]
        lower = t_down[:,:,a,:]
        @tensor upperU[i,i2,j1,k] := u_left[i1,i2,i] * upper[i1,j1,k]
        @tensor lowerU[i2,j,j1,l] := u_right[j1,j2,j] * lower[i2,j2,l]
        @tensor temp[i,j,k,l] := upperU[i,i2,j1,k] * lowerU[i2,j,j1,l]
        res += temp        
    end
    return res
end;

In [8]:
"""
          3                1
          |                |
      1 --o-- 2   -->  4 --o-- 3
          |                |
          4                2
"""
function tensor_rotate_clockwise(ten)
    return permutedims(ten, (4, 3, 1, 2))
end;

In [9]:
function calculate_free_energy_2d(temperature, list_of_norms)
    rv = 0.
    for i in 1:length(list_of_norms)
        rv += log(list_of_norms[i]) / (2.0 ^ (i - 1))
    end
    return - temperature * rv
end;

In [10]:
function calculate_entanglement_entropy(s)
    v = s / sum(s)
    entr = 0.
    for x in v
        if x > 1.e-14
            entr += x * log(x)
        end
    end    
    return -entr
end;

In [11]:
function tensor_trace(ten)    
    @einsum ten_t := ten[i,i,j,j]
    return ten_t
end;

function calculate_expectation(ten, imp)
    return tensor_trace(imp) / tensor_trace(ten)
end;

Calculating converged free energy per site, magnetization, entanglement entropy (not converged) for given temperature, external field $h$, and bond dimension.

In [12]:
function hotrg(file_iter_name, temp, h, dim)
    open(file_iter_name, "a") do f
        write(f, "# temp=$(temp)\n")
        write(f, "# iter\tfree energy\t\tmag\t\t\tentropy\n")
    end
    ten = initialize_bulk_tensor(temp, h)
    imp = initialize_imp_t(temp, h)
    list_of_norms = []
    norm = maximum(abs, ten)
    push!(list_of_norms, norm)
    ten = ten / norm
    imp = imp / norm
    free_energy = 0
    free_energy_new = -1
    mag = 0.
    mag_new = -1.
    entanglement_entropy = 0
    iter_count = 0
    while abs(free_energy - free_energy_new) > 1.e-14 || abs(mag - mag_new) > 1.e-14
        u, ss = create_hotrg_projector_2d(dim, ten, ten)        
        imp = (update_optimized(imp, ten, u, u) + update_optimized(ten, imp, u, u)) / 2
        ten = update_optimized(ten, ten, u, u)
        norm = maximum(abs, ten)
        push!(list_of_norms, norm)
        ten = ten / norm
        imp = imp / norm
        free_energy = free_energy_new
        free_energy_new = calculate_free_energy_2d(temp, list_of_norms)
        mag = mag_new
        mag_new = calculate_expectation(ten, imp)
        entanglement_entropy = calculate_entanglement_entropy(ss)
        open(file_iter_name, "a") do f
            write(f, @sprintf("%d\t%.15f\t%.15f\t%.15f\n", iter_count, free_energy_new, mag_new, entanglement_entropy))
        end
        ten = tensor_rotate_clockwise(ten)
        imp = tensor_rotate_clockwise(imp)
        iter_count += 1
    end
    return free_energy_new, mag_new, entanglement_entropy, iter_count
end;

Finally, we will calculate the profile of free energy, magnetization, and entanglement entropy w.r.t. temperature in a given range. 

In [13]:
dim = 10
start_val = 2.
end_val = 2.5
num = 51
field = 1.e-14

file_name = "data_dim$(dim).txt"
file_iter_name = "data_iter_dim$(dim).txt"

open(file_name, "w") do f
    write(f, "# D=$(dim), h=$(field)\n")
    write(f, "# temp\t\t\tfree energy\t\tmag\t\t\tentropy\t\t\titer\n")
end

open(file_iter_name, "w") do f
    write(f, "# D=$(dim), h=$(field)\n")
end

for temperature in range(start_val, stop=end_val, length=num)
    free_energy, mag, entanglement_entropy, iter_count = hotrg(file_iter_name, temperature, field, dim)
    open(file_name, "a") do f
        write(f, @sprintf("%.15f\t%.15f\t%.15f\t%.15f\t%d\n", temperature, free_energy, mag, entanglement_entropy, iter_count))
    end
end